In [45]:
import os
import pickle
import PIL.Image
import numpy as np
import pandas as pd
import sys

# path to stylegan encoder locally
sys.path.insert(1, '/home/bobby/APhotoADay/data/stylegan-encoder')

import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

ValueError: Variable learnable_dlatents already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/bobby/APhotoADay/data/stylegan-encoder/encoder/generator_model.py", line 15, in create_variable_for_generator
    initializer=tf.initializers.random_normal())
  File "/home/bobby/APhotoADay/data/stylegan-encoder/dnnlib/tflib/network.py", line 406, in <listcomp>
    in_expr = [input_builder(name) for input_builder, name in zip(custom_inputs, self.input_names)]
  File "/home/bobby/APhotoADay/data/stylegan-encoder/dnnlib/tflib/network.py", line 406, in run
    in_expr = [input_builder(name) for input_builder, name in zip(custom_inputs, self.input_names)]
  File "/home/bobby/APhotoADay/data/stylegan-encoder/encoder/generator_model.py", line 27, in __init__
    structure='fixed')
  File "<ipython-input-5-5b8e6e491508>", line 7, in <module>
    generator = Generator(Gs_network, batch_size=1, randomize_noise=False)


In [47]:
def generate_image(latent_vector, size):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

def show(latent_vector):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    ax[i].imshow(generate_image(latent_vector))
    ax[i].set_title(latent_vector)
    [x.axis('off') for x in ax]
    plt.show()

In [48]:
# Receive input

real_data_path = "real_data"#input("Enter path for real data: ")
fake_latent_path = "fake_latent"#input("Enter path for fake latent: ")
start_age = int(input("Enter start age (days): "))
end_age = int(input("Enter end age (days): "))
num_of_vid = "2"#input("Enter video number: ")

experiment = "video" + num_of_vid + "_plot_" + str(start_age) + "_to_" + str(end_age)

Enter start age (days): 5840
Enter end age (days): 7940


In [49]:
# Decode fakes

fake_data_path = "fake_data"#input("Enter path for fake data: ")

if not os.path.exists(fake_data_path):
    os.mkdir(fake_data_path)

for file in os.listdir(fake_latent_path):
    img = generate_image(np.load(fake_latent_path + "/" + file), 1024)
    img.save(fake_data_path + "/" + file[:-4] + ".png")

In [ ]:
# Display GIFs

def gif_create(delay, path_to_frames, output):
    os.system("convert -delay " + delay + " -loop 0 " + path_to_frames + "/*.png Results/" + experiment + "/" + output + ".gif")

gif_create("10", real_data_path, "real")
gif_create("10", fake_data_path, "fake")